In [3]:
from data_parser import DataParser
from datetime import datetime
import pandas as pd
import numpy as np
import copy

In [4]:
dp = DataParser(filename='data/GBPUSD.csv')

In [5]:
start_datetime = datetime.strptime('20021001 223600','%Y%m%d %H%M%S')
end_datetime = datetime.strptime('20120228 183600','%Y%m%d %H%M%S')

In [6]:
dp.filter_by_date(start_datetime, end_datetime)

,open,high,low,close,vol
date,,,,,
2002-10-01 22:37:00,1.5617,1.5617,1.5612,1.5613,4
2002-10-01 22:38:00,1.5614,1.5614,1.5612,1.5612,4
2002-10-01 22:39:00,1.5614,1.5614,1.5614,1.5614,4
2002-10-01 22:40:00,1.5613,1.5614,1.5613,1.5614,4
2002-10-01 22:41:00,1.5613,1.5613,1.5613,1.5613,4
...,...,...,...,...,...
2012-02-28 18:32:00,1.5879,1.5879,1.5878,1.5879,4
2012-02-28 18:33:00,1.5880,1.5882,1.5880,1.5880,4
2012-02-28 18:34:00,1.5880,1.5880,1.5880,1.5880,4


In [7]:
dp.sample_by_time('40min', 2160000000000)

,date,open,high,low,close,vol
0,2002-10-01 22:36:00,1.5617,1.5619,1.5612,1.5618,76
1,2002-10-01 23:16:00,1.5618,1.5624,1.5615,1.5621,84
2,2002-10-01 23:56:00,1.5623,1.5623,1.5609,1.5620,92
3,2002-10-02 00:36:00,1.5621,1.5621,1.5616,1.5619,68
4,2002-10-02 01:16:00,1.5618,1.5619,1.5610,1.5610,64
...,...,...,...,...,...,...
123722,2012-02-28 15:56:00,1.5828,1.5899,1.5825,1.5890,160
123723,2012-02-28 16:36:00,1.5891,1.5893,1.5879,1.5882,160
123724,2012-02-28 17:16:00,1.5881,1.5881,1.5873,1.5875,160
123725,2012-02-28 17:56:00,1.5875,1.5887,1.5875,1.5879,160


In [8]:

Xn,yn = dp.get_close_price_numpy_array(1, 22, 21, 30,36)

In [9]:
Xn1 = dp.tranform_X_to_returns(Xn)

In [10]:
Xn2 = dp.transform_X_with_IWMA(Xn1)

In [11]:
X_norm, y_norm = dp.normalize_data(Xn2, yn)

In [12]:
print(X_norm.shape)
print(y_norm.shape)

(2429, 20)
(2429,)


In [13]:
from sklearn.model_selection import train_test_split

In [14]:
train_X, test_X, train_y, test_y = train_test_split(X_norm, y_norm, test_size=0.2, random_state=42)
train_X_in, test_X_in, train_y_in, test_y_in = train_test_split(train_X, train_y, test_size=0.1, random_state=42)

In [15]:
from ann import ANN

In [16]:
from GA import GeneticAlgorithm

In [22]:
def fitness(chromosome):
    #chromosomes have 8 bits. first 4 bits are the hidden layer 1, next 4 bits are hidden layer 2. but chromosome is a numpy array
    hidden_layer_1 = 0
    hidden_layer_2 = 0
    for i in range(4):
        hidden_layer_1 += chromosome[i] * (2 ** (3-i))
        hidden_layer_2 += chromosome[i+4] * (2 ** (3-i))
#take mod with 20
    hidden_layer_1 = (hidden_layer_1 % 20) + 1
    hidden_layer_2 = (hidden_layer_2 % 20) + 1
    ann = ANN(hidden_layer_1, hidden_layer_2, 20, 1, 'tanh','lbfgs', 2000,42)
    ann.fit(train_X_in, train_y_in)
    y_pred = ann.predict(test_X_in)
    nc = ann.num_correct(test_y_in, y_pred)
    return nc

In [23]:
ga = GeneticAlgorithm(population_size=30, chromosome_length=8, crossover_rate=0.8, mutation_rate=0.2, num_generations=1, stall_generations=15)
ga.fitness_function = fitness
best_chromosome, best_fitness = ga.run()

Generation 1 of 1


/home/slowblow/miniconda3/envs/algoedge/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/slowblow/miniconda3/envs/algoedge/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/slowblow/miniconda3/envs/algoedge/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: Conv

Best Chromosome: [1 0 1 1 1 1 1 1]
Best Fitness: 108


/home/slowblow/miniconda3/envs/algoedge/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [19]:
print(best_chromosome)

[1 0 0 1 1 1 1 1]


In [20]:
hidden_layer_1 = 0
hidden_layer_2 = 0
for i in range(4):
    hidden_layer_1 += best_chromosome[i] * (2 ** (3-i))
    hidden_layer_2 += best_chromosome[i+4] * (2 ** (3-i))
#take mod with 20
hidden_layer_1 = (hidden_layer_1 % 20) + 1
hidden_layer_2 = (hidden_layer_2 % 20) + 1
ann = ANN(hidden_layer_1, hidden_layer_2, 20, 1, 'tanh', 'adam', 100, 42)
ann.fit(train_X_in, train_y_in)
y_pred = ann.predict(test_X)
nc = ann.num_correct(test_y, y_pred)
print(nc)

238


In [21]:
print(ann.num_wrong(test_y, y_pred))

248
